# Adidas sales data analysis <br/>
## Business Metrics requirements <br/>
1. Total sales, total profit, average price per unit, total units sold
2. total sales by month
3. total sales by state
4. total sales by region
5. total sales by product
6. total sales by retailer
7. units sold by product category and gender type
8. top performing cities by profit

In [0]:
from pyspark.sql import SparkSession

df = spark.read.csv('/FileStore/tables/Adidas_US_Sales_Datasets.csv', header=True, inferSchema=True)
display(df.limit(5))

Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
Foot Locker,1185732,2020-01-01,Northeast,New York,New York,Men's Street Footwear,50.0,"1,200","600,000","300,000",50%,In-store
Foot Locker,1185732,2020-01-02,Northeast,New York,New York,Men's Athletic Footwear,50.0,"1,000","500,000","150,000",30%,In-store
Foot Locker,1185732,2020-01-03,Northeast,New York,New York,Women's Street Footwear,40.0,"1,000","400,000","140,000",35%,In-store
Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,45.0,850,"382,500","133,875",35%,In-store
Foot Locker,1185732,2020-01-05,Northeast,New York,New York,Men's Apparel,60.0,900,"540,000","162,000",30%,In-store


In [0]:
df.createOrReplaceTempView('adidas_sales')

In [0]:
%sql
select * from adidas_sales limit 5

Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
Foot Locker,1185732,2020-01-01,Northeast,New York,New York,Men's Street Footwear,50.0,"1,200","600,000","300,000",50%,In-store
Foot Locker,1185732,2020-01-02,Northeast,New York,New York,Men's Athletic Footwear,50.0,"1,000","500,000","150,000",30%,In-store
Foot Locker,1185732,2020-01-03,Northeast,New York,New York,Women's Street Footwear,40.0,"1,000","400,000","140,000",35%,In-store
Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,45.0,850,"382,500","133,875",35%,In-store
Foot Locker,1185732,2020-01-05,Northeast,New York,New York,Men's Apparel,60.0,900,"540,000","162,000",30%,In-store


In [0]:
#Total sales, total profit, average price per unit, total units sold

display(
spark.sql("""\
    select sum(`Total Sales`) as Total_Sales, sum(`Operating Profit`) as Total_Profit,
    round(avg(`Price per Unit`),2) as Avg_Price_Per_Unit, sum(`Units Sold`) as Total_Units_Sold
    from adidas_sales
    """))

Total_Sales,Total_Profit,Avg_Price_Per_Unit,Total_Units_Sold
107905.0,635412.0,45.22,2421491.0


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import *
df.agg(sum('Total Sales').alias('Total Sales'),\
        sum('Operating Profit').alias('Total Profit'),\
        round(avg('Price per Unit'),2).alias('Avg Price per Unit'),\
        sum('Units Sold').alias('Total Units Sold')).show()

+-----------+------------+------------------+----------------+
|Total Sales|Total Profit|Avg Price per Unit|Total Units Sold|
+-----------+------------+------------------+----------------+
|   107905.0|    635412.0|             45.22|       2421491.0|
+-----------+------------+------------------+----------------+



In [0]:
#total sales by month

display(\
spark.sql("""\
    select sum(`Total Sales`) as Total_Sales, date_format(`Invoice Date`,'MMMM') as Month from adidas_sales
    group by 2
    order by 1 desc
    """))

Total_Sales,Month
20611.0,March
20135.0,April
13056.0,February
11465.0,September
10983.0,October
8142.0,November
6602.0,January
5511.0,June
5060.0,July
3979.0,December


Databricks visualization. Run in Databricks to view.

In [0]:
display(df.groupBy(date_format('Invoice Date','MMMM').alias('Month')).agg(sum('Total Sales').alias('total sales')))

Month,total sales
July,5060.0
November,8142.0
February,13056.0
January,6602.0
March,20611.0
October,10983.0
May,2361.0
August,null
April,20135.0
June,5511.0


In [0]:
#total sales by state

display(\
spark.sql("""\
    select sum(`Total Sales`) as Total_Sales, State from adidas_sales 
    group by 2
    order by 1 desc
    limit 5
    """))

Total_Sales,State
13273.0,Oklahoma
12609.0,Rhode Island
9318.0,Nebraska
7740.0,Minnesota
6770.0,Iowa


Databricks visualization. Run in Databricks to view.

In [0]:
display(df.groupBy('State').agg(sum('Total Sales').alias('total_sales')).orderBy(col('total_sales').desc()).limit(5))

State,total_sales
Oklahoma,13273.0
Rhode Island,12609.0
Nebraska,9318.0
Minnesota,7740.0
Iowa,6770.0


In [0]:
#total sales by region

display(\
spark.sql("""\
    select sum(`Total Sales`) as Total_Sales, Region from adidas_sales
    group by 2
    order by 1 desc
    limit 5
    """))

Total_Sales,Region
47554.0,Midwest
35316.0,Northeast
20098.0,South
3211.0,West
1726.0,Southeast


Databricks visualization. Run in Databricks to view.

In [0]:
display(df.groupBy('Region').agg(sum('Total Sales').alias('total_sales')).orderBy(col('total_sales').desc()).limit(5))

Region,total_sales
Midwest,47554.0
Northeast,35316.0
South,20098.0
West,3211.0
Southeast,1726.0


In [0]:
#total sales by product

display(\
spark.sql("""\
    select sum(`Total Sales`) as total_sales, Product from adidas_sales
    group by 2
    order by 1 desc
    limit 5
    """))

total_sales,Product
62033.0,Women's Athletic Footwear
23104.0,Women's Street Footwear
13883.0,Men's Apparel
7023.0,Men's Athletic Footwear
1862.0,Women's Apparel


Databricks visualization. Run in Databricks to view.

In [0]:
display(df.groupBy('Product').agg(sum('Total Sales').alias('total_sales')).orderBy(col('total_sales').desc()).limit(5))

Product,total_sales
Women's Athletic Footwear,62033.0
Women's Street Footwear,23104.0
Men's Apparel,13883.0
Men's Athletic Footwear,7023.0
Women's Apparel,1862.0


In [0]:
#total sales by retailer

display(\
spark.sql("""\
    select sum(`Total Sales`) as total_sales,Retailer from adidas_sales
    group by 2
    order by 1 desc
    limit 5
    """))

total_sales,Retailer
50664.0,Foot Locker
30663.0,West Gear
10907.0,Sports Direct
8526.0,Kohl's
5369.0,Amazon


Databricks visualization. Run in Databricks to view.

In [0]:
display(df.groupBy('Retailer').agg(sum('Total Sales').alias('total_sales')).orderBy(col('total_sales').desc()).limit(5))

Retailer,total_sales
Foot Locker,50664.0
West Gear,30663.0
Sports Direct,10907.0
Kohl's,8526.0
Amazon,5369.0


In [0]:
#units sold by product category and gender type

display(\
spark.sql("""\
    select sum(`Units Sold`) as total_units_sold, Product from adidas_sales
    group by 2 limit 5
    """))

total_units_sold,Product
429476.0,Men's Athletic Footwear
317236.0,Women's Athletic Footwear
306683.0,Men's Apparel
423602.0,Women's Apparel
389069.0,Women's Street Footwear


Databricks visualization. Run in Databricks to view.

In [0]:
display(df.groupBy('Product').agg(sum('Total Sales').alias('total_sales')).orderBy(col('total_sales').desc()).limit(5))

Product,total_sales
Women's Athletic Footwear,62033.0
Women's Street Footwear,23104.0
Men's Apparel,13883.0
Men's Athletic Footwear,7023.0
Women's Apparel,1862.0


In [0]:
#top performing cities by profit

display(\
spark.sql("""\
    select sum(`Operating Profit`) as Total_Profit, City from adidas_sales
    group by 2
    order by 1 desc
    limit 5
    """))

Total_Profit,City
40190.0,Providence
37393.0,Philadelphia
35842.0,Boston
31313.0,Portland
30809.0,Salt Lake City


Databricks visualization. Run in Databricks to view.

In [0]:
#units sold by sales method

display(\
    spark.sql("""\
        select sum(`Units Sold`) as total_units_sol, `Sales Method` from adidas_sales
        group by 2
        order by 1
        limit 5
        """)
    )

total_units_sol,Sales Method
672750.0,In-store
821313.0,Outlet
927428.0,Online


Databricks visualization. Run in Databricks to view.